In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from utils.preprocessing import create_time_matrix, create_caregivers_df, create_tasks_df, create_clients_df

caregivers = create_caregivers_df('data/raw/caregivers.csv')

tasks = create_tasks_df('data/raw/hemtjanst_tasks.csv', only_client_tasks=True)

clients = create_clients_df('data/raw/clients.csv')

drive_time_matrix = create_time_matrix('data/raw/matrix_duration_matrix_DRIVE.csv')
walk_time_matrix = create_time_matrix('data/raw/matrix_duration_matrix_WALK.csv')
bicycle_time_matrix = create_time_matrix('data/raw/matrix_duration_matrix_BICYCLE.csv')

In [2]:
# Test1: Subset with only caregivers 33, 949, 1108 and their tasks
test1_subset = [33, 949, 1108]
caregivers_test1 = caregivers.loc[test1_subset]
tasks_test1 = tasks[tasks['PlannedCaregiverID'].isin(test1_subset)]

print(f"Number of caregivers in subset: {len(caregivers_test1)}")
print(f"Number of tasks assigned to subset caregivers: {len(tasks_test1)}")

# Test2: Subset with only first 10 caregivers that use car as mode of transport
car_caregivers = caregivers[caregivers['ModeOfTransport'] == 'car']
caregivers_test2 = car_caregivers.head(10)
tasks_test2 = tasks[tasks['PlannedCaregiverID'].isin(caregivers_test2.index)]

print(f"Number of caregivers in subset: {len(caregivers_test2)}")
print(f"Number of tasks assigned to subset caregivers: {len(tasks_test2)}")

Number of caregivers in subset: 3
Number of tasks assigned to subset caregivers: 27
Number of caregivers in subset: 10
Number of tasks assigned to subset caregivers: 86


In [ ]:
from models.flexible_model import FlexibleModel
from models.fixed_model import FixedModel
model_type = FixedModel
model_caregivers = caregivers_test2
model_tasks = tasks_test2

model = model_type(model_caregivers, model_tasks, clients, drive_time_matrix, walk_time_matrix, bicycle_time_matrix)
model.build()
model.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.3.0 24D70)

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 988 rows, 76540 columns and 266490 nonzeros
Model fingerprint: 0xc6863fde
Variable types: 1720 continuous, 74820 integer (74820 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 42 rows and 43421 columns
Presolve time: 0.12s
Presolved: 946 rows, 33119 columns, 96957 nonzeros
Variable types: 0 continuous, 33119 integer (33099 binary)

Root relaxation: objective 3.803650e+03, 14859 iterations, 1.20 seconds (2.90 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    3803.6500000 3803.65000  0.00%     -    1s



<gurobi.Model MIP instance HomeCare: 988 constrs, 76540 vars, Parameter changes: Username=(user-defined), LicenseID=2635249>

In [ ]:
from utils.visualization import visualize_schedule, visualize_metrics, visualize_routes

model.get_solution()

visualize_metrics(model, display_mode='dashboard');
visualize_schedule(model);
visualize_routes(model, subplot_mode=True);

KeyError: 979